# 数据清理

## URL 合并

In [1]:
# 导入数据文件

# here put the import lib
import os

import logging as log
import pandas as pd

from modules.logger.logger import init as logInit
from modules.logUtils.httpEvent import HttpEvent, TABLE_LABELS
from modules.fileUtils.fileUtils import getFilesByPath
from modules.config.argumentParser import argumentParser
from modules.config.configParser import getValue as getValueFromConfig

# CONFIG

TABLE_LABELS = [
    "timestamp",
    "threadId",
    "institutionId",
    "userId",
    "url",
    "method",
    "statusCode",
    "parameterType",
    "parameterName",
    "parameterValue",
    "headers",
    "name",
    "ip",
    "port",
    "vpnip"
]
# get all files.
files = getFilesByPath("data/origin")
if not files: 
    print("there no any file. recorrect the input path.")
files = files

import random

# 随机抽取 RANDOM_FILES_NUMBER 数量的日志
RANDOM_FILES_NUMBER = 0
files = random.sample(files, RANDOM_FILES_NUMBER) if RANDOM_FILES_NUMBER != 0 else files

print(len(files))
# print(files)

994


In [ ]:
# 将数据读取到 DataFrame 中

data = []
for file in files:
    count = 0
    with open(file, 'r', encoding='utf-8') as f:
        # read raw log by line.
        for line in f:
            try:
                he = HttpEvent(line)
                
                # filter user
                if he.userId == 'null': continue

                # append to Data
                data.extend([he.getSet()])
                count += 1
            except Exception as e:
                log.debug(e)
    log.info("{0} write {1} lines".format(file, count))
print(len(data), len(data[0]))
# convert List to DataFrame
# print(data[0])
df = pd.DataFrame(data = data, columns=TABLE_LABELS)
del data

In [ ]:
# 保存到 CSV
df.to_csv("data/tmp/full.csv", sep="^")
!ls -l data/tmp/full.csv

In [ ]:
# 从 CSV 读取
import pandas as pd
df = pd.read_csv('data/tmp/full.csv', sep='^')

In [ ]:
# 查看机构号
institutesDF = df.institutionId
institutes = institutesDF.value_counts()
for k, v in institutes.items():
    print(v, k)

In [ ]:
# 查看 userId
userIdDF = df.userId
users = userIdDF.value_counts()
for k, v in users.items():
    print(v, k)
print(len(users))

In [ ]:
# 输出所有 url 和次数
urlDF = df.url
urls = urlDF.value_counts()

# for k, v in urls.items():
#     print(v, k)

print(len(urls))

In [ ]:
# convert all urls
from modules.logUtils.urlFilter import urlConvert
df['url'] = df['url'].apply(urlConvert, args=(set(df.userId.values), set(df.institutionId.values),  set(df.name.values),))
# save to CSV
df.to_csv("data/tmp/converedtUrl_full.csv", sep='^')

In [ ]:
# 输出所有 url 和次数
urlDF = df.url
urls = urlDF.value_counts()

# for k, v in urls.items():
#     print(v, k)

print(len(urls))

In [ ]:
# delete bug log
df = df.drop(index=(df.loc[(df.name=='何群辉')].index))
# delete unused log
df = df.drop(index=(df.loc[(df.header=="null")].index))
# delete unused column
df = df.drop(['threadId', 'statusCode', 'parameterType'], axis=1)

In [ ]:
# Sort by userId primary key, with time as the secondary key
df.sort_values(by=['userId', 'timestamp'])

In [ ]:
df.to_csv("data/tmp/baseDataSet.csv", sep="^")

## URL 映射表制作

In [ ]:
urlSet = set(df.url.values)
urlToIndex = {url:i.__str__() for i, url in enumerate(urlSet)}
urlIndexToUrl = {i.__str__():url for i, url in enumerate(urlSet)}
print(len(urlToIndex), len(urlIndexToUrl))
# print(urlIndexToUrl)
import json
import os
if not os.path.exists('data/url'):
    os.makedirs('data/url')
    
def writeJson(filename, dic):
    with open(filename,'a', encoding="utf-8") as outfile:
        json.dump(dic,outfile,ensure_ascii=False)

writeJson("data/url/urlToIndex.json", urlToIndex)
writeJson("data/url/urlIndexToUrl.json", urlIndexToUrl)

In [ ]:
# read json to dict
def readJson(filename):
    with open(filename, "r", encoding="utf-8") as target:
        return json.load(target)
x = readJson('data/url/urlToIndex.json')
print(len(x))

## 会话识别

In [ ]:
def generateSession(hes: pd.DataFrame):
    #########################################################
    # TODO: simply save the index of url, and drop others.
    #########################################################
    
    sessions = []
    
    # read by line
    paths = []
    lastHeaderKey = ""
    lastHEDate = 0.0
    threshold = 0
    print(type(hes.timestamp))
    
#     for he in hes.itertuples():
#         #  get session key in headers       
#         headersJSON = json.loads(he.header)
#         flag = False
#         for header in headersJSON:
#             flag = True if header['value']==lastHeaderKey else False
            
#         if not flag and float(he.timestamp) - lastHEDate > threshold:
#             # start a new session
#             if paths:
#                 sessions.append("-".join([urlToIndex[url] for url in paths]))
#             paths.clear()
#         ############
#         print(he.timestamp, he.header, float(he.timestamp) - lastHEDate)
#         ############
            
#         # update paths 
#         paths.append(he.url)
#         lastHeaderKey = headersJSON[0]['value']
#         lastHEDate = float(he.timestamp)
#     sessions.append("-".join([urlToIndex[url] for url in paths]))
    return sessions

        
for userId, indexs in df.groupby('userId').groups.items():
    tmpSessions = generateSession(df.loc[indexs])
    print(tmpSessions)
    break

In [ ]:
headers = '''[{"httpOnly":false,"maxAge":-1,"name":"Authorization","secure":false,"value":"aTrSU0S4j6YQ","version":0}, {"httpOnly":false,"maxAge":-1,"name":"Authorization","secure":false,"value":"aTrSU0S4j654","version":0}]'''
# print(headers)
x = json.loads(headers)
print(x[0]['value'])


In [ ]:
paths = ['/fa/commoncore/todoCommonCore/remoteGetTotalTodoNum', '/portal/shortcutconfig/getConfigCompletion', '/portal/menus/mine/pc', '/portal/menus', '/portal/shortcutconfig/getConfigCompletion', '/portal/menus', '/portal/userusages', '/portal/menus/url', '/portal/dictionary/codes/code/code', '/portal/dictionary/codes/code/code', '/portal/dictionary/codes/code/code', '/portal/dictionary/codes/code/code', '/portal/menus/mine/pc', '/portal/shortcutconfig/getConfigCompletion', '/portal/menus/url', '/portal/dictionary/codes/code/code', '/portal/dictionary/codes/code/code', '/portal/dictionary/codes/code/code', '/portal/menus', '/portal/dictionary/codes/code/code', '/portal/userusages', '/pm/commoncore/todoCommonCore/getPmTotalPendingCount', '/od/workflow/getToDoList', '/od/tOdQpjOrgids/querySignProcess', '/od/tOdArchivesOutOrgids/getByOutTurnover', '/od/tOdArchivesOutOrgids/getByDocumentTypeTwo', '/od/tOdQpjOrgids/getBySignType', '/od/tOdArchivesInOrgids/queryInProcess', '/od/tOdArchivesInOrgids/getByInTypeCount', '/od/workflow/getToDoList', '/od/workflow/getAllTodoNumber', '/od/workflow/getToDoList', '/od/arp24Controller/getSsoURL', '/od/odRolePromissConfig/findAll', '/od/tOdFlowDefined/findAllValidChildByOdType', '/od/tOdQpjOrgids/getQueryList', '/od/arp24Controller/getSsoURL', '/od/odRolePromissConfig/findAll', '/od/tOdFlowDefined/findAllValidChildByOdType', '/od/tOdQpjOrgids/getQueryList', '/hr/thrpeople/', '/fin/arBill/getArBillAllTODOCount']
# for url in paths:
#     print(url)

pathss = "-".join([urlToIndex[url] for url in paths])
print(pathss)

In [ ]:
['441-237', '1298', '463', '237', '463', '540', '1017', '346', '346-346', '346', '1298', '237', '1017', '346', '346', '346', '463-346', '540-1537-409-771-197-207-1199-1018-594-409-1013-409-823-110-691-1217-823-110-691-1217-1341-6']

In [ ]:
print()